# BEDBASE workflow tutorial

The following demo has the purpose of demonstrating how to process, generate statistics and plots of BED files generated by the R package Genomic Distributions using the `bedhost` REST API for the bedstat and bedbuncher pipelines output. 

Notes:

- If this hasn't been already done, we recommend starting this jupyter notebook enabling sudo permissions since steps such as downloading `docker` or running an elasticsearch `docker` container won't be executed otherwise. This can be done with `sudo jupyter notebook --allow-root`

 
 

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Create-a-tutorial-directory-and-download-demo-files" data-toc-modified-id="Create-a-tutorial-directory-and-download-demo-files-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Create a tutorial directory and download demo files</a></span></li><li><span><a href="#Generate-statistics-and-plots-of-BED-files-using-BEDSTAT" data-toc-modified-id="Generate-statistics-and-plots-of-BED-files-using-BEDSTAT-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Generate statistics and plots of BED files using BEDSTAT</a></span><ul class="toc-item"><li><span><a href="#Create-a-PEP-describing-the-BED-files-to-process" data-toc-modified-id="Create-a-PEP-describing-the-BED-files-to-process-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Create a PEP describing the BED files to process</a></span></li><li><span><a href="#Download-bedstat-and-the-Bedbase-configuration-manager-(bbconf)" data-toc-modified-id="Download-bedstat-and-the-Bedbase-configuration-manager-(bbconf)-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Download bedstat and the Bedbase configuration manager (bbconf)</a></span></li><li><span><a href="#Inititiate-a-local-elasticsearch-cluster" data-toc-modified-id="Inititiate-a-local-elasticsearch-cluster-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Inititiate a local elasticsearch cluster</a></span></li><li><span><a href="#Run-bedstat--on-the-demo-PEP" data-toc-modified-id="Run-bedstat--on-the-demo-PEP-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Run bedstat  on the demo PEP</a></span></li></ul></li><li><span><a href="#Create-bedsets-using-BEDBUNCHER" data-toc-modified-id="Create-bedsets-using-BEDBUNCHER-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Create bedsets using BEDBUNCHER</a></span><ul class="toc-item"><li><span><a href="#Create-a-new-PEP-describing-the-bedset-name-and-specific-JSON-query" data-toc-modified-id="Create-a-new-PEP-describing-the-bedset-name-and-specific-JSON-query-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Create a new PEP describing the bedset name and specific JSON query</a></span></li><li><span><a href="#Download-bedbuncher--and-install-CML-dependencies" data-toc-modified-id="Download-bedbuncher--and-install-CML-dependencies-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Download bedbuncher  and install CML dependencies</a></span></li><li><span><a href="#Run-bedbuncher-using-Looper" data-toc-modified-id="Run-bedbuncher-using-Looper-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Run bedbuncher using Looper</a></span></li></ul></li><li><span><a href="#Run-local-instance-of-the-bedhost-API" data-toc-modified-id="Run-local-instance-of-the-bedhost-API-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Run local instance of the bedhost API</a></span></li></ul></div>

## Create a tutorial directory and download demo files 
We need create a directory where we'll store the bedbase pipelines and files to be processed. We'll also need to create an environment variable that points to the tutorial directory (we'll need this variable in section 3 of the tutorial). 

In [1]:
cd $HOME

In [2]:
mkdir bedbase_tutorial
cd bedbase_tutorial
export BEDBASEtutorial="$HOME/bedbase_tutorial"
source ~/.bashrc

To download the files we'll need for this tutorial, we can easily do it with the following commands:

In [3]:
wget http://big.databio.org/example_data/bedbase_demo/bedbase_demo_files_justBED/bedbase_BEDfiles.tar.gz     
wget http://big.databio.org/example_data/bedbase_demo/bedbase_demo_files_justBED/bedbase_demo_PEPs.tar.gz 

--2020-03-19 16:57:03--  http://big.databio.org/example_data/bedbase_demo/bedbase_demo_files_justBED/bedbase_BEDfiles.tar.gz
Resolving big.databio.org (big.databio.org)... 128.143.245.181
Connecting to big.databio.org (big.databio.org)|128.143.245.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60245813 (57M) [application/octet-stream]
Saving to: ‘bedbase_BEDfiles.tar.gz’

bedbase_BEDfiles.ta 100%[===================>]  57.45M   914KB/s    in 59s     

2020-03-19 16:58:02 (993 KB/s) - ‘bedbase_BEDfiles.tar.gz’ saved [60245813/60245813]

--2020-03-19 16:58:02--  http://big.databio.org/example_data/bedbase_demo/bedbase_demo_files_justBED/bedbase_demo_PEPs.tar.gz
Resolving big.databio.org (big.databio.org)... 128.143.245.181
Connecting to big.databio.org (big.databio.org)|128.143.245.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1374 (1.3K) [application/octet-stream]
Saving to: ‘bedbase_demo_PEPs.tar.gz’

bedbase_demo_PEPs.t 100%

The downloaded files are compressed so we'll need to untar them:

In [4]:
tar -zxvf bedbase_BEDfiles.tar.gz
tar -zxvf bedbase_demo_PEPs.tar.gz

bedbase_BEDfiles/
bedbase_BEDfiles/GSE105977_ENCFF449EZT_optimal_idr_thresholded_peaks_hg19.bed.gz
bedbase_BEDfiles/GSE105587_ENCFF018NNF_conservative_idr_thresholded_peaks_GRCh38.bed.gz
bedbase_BEDfiles/GSE105587_ENCFF413ANK_peaks_hg19.bed.gz
bedbase_BEDfiles/GSM2423312_ENCFF155HVK_peaks_GRCh38.bed.gz
bedbase_BEDfiles/GSE105977_ENCFF617QGK_optimal_idr_thresholded_peaks_GRCh38.bed.gz
bedbase_BEDfiles/GSE91663_ENCFF316ASR_peaks_GRCh38.bed.gz
bedbase_BEDfiles/GSM2423313_ENCFF722AOG_peaks_GRCh38.bed.gz
bedbase_BEDfiles/GSE105587_ENCFF809OOE_conservative_idr_thresholded_peaks_hg19.bed.gz
bedbase_BEDfiles/GSM2827349_ENCFF196DNQ_peaks_GRCh38.bed.gz
bedbase_BEDfiles/GSE91663_ENCFF553KIK_optimal_idr_thresholded_peaks_GRCh38.bed.gz
bedbase_BEDfiles/GSE91663_ENCFF319TPR_conservative_idr_thresholded_peaks_GRCh38.bed.gz
bedbase_BEDfiles/GSE105977_ENCFF634NTU_peaks_hg19.bed.gz
bedbase_BEDfiles/GSE105977_ENCFF937CGY_peaks_GRCh38.bed.gz
bedbase_BEDfiles/GSM2827350_ENCFF928JXU_peaks_GRCh38.bed.gz
bedb

## Generate statistics and plots of BED files using BEDSTAT


### Create a PEP describing the BED files to process

In order to get started, we'll need a PEP [Portable Encapsulated project](https://pepkit.github.io/). A PEP consists of 1) an annotation sheet (.csv) that contains information about the samples on a project and 2) a project config.yaml file that points to the sample annotation sheet. The config file also has other components, such as derived attributes, that in this case point to the BED files to be processed. The following is an example of a config file using the derived attributes `output_file_path` and `yaml_file` to point to the `.bed.gz` files and their respective metadata.

In [5]:
cat bedbase_demo_PEPs/bedstat_config.yaml

metadata:
  sample_table: bedstat_annotation_sheet.csv
  output_dir: ../bedstat/bedstat_pipeline_logs 
  pipeline_interfaces: ../bedstat/pipeline_interface.yaml

constant_attributes: 
  output_file_path: "source"
  yaml_file: "source2"
  protocol: "bedstat"

derived_attributes: [output_file_path, yaml_file]
data_sources:
  source: ../bedbase_BEDfiles/{file_name} 
  source2: ../bedstat/bedstat_pipeline_logs/submission/{sample_name}.yaml


### Download bedstat and the Bedbase configuration manager (bbconf)

[bedstat](https://github.com/databio/bedstat) is a [pypiper](http://code.databio.org/pypiper/) pipeline that generates statistics and plots of BED files. Additionally, [bedstat](https://github.com/databio/bedstat) relies in
[bbconf](https://github.com/databio/bbconf), the `bedbase` configuration manager which implements convenience methods for interacting with an elasticsearch database, where our files metadata will be placed. For carrying out this demo, we'll be using the dev version of `bbconf` that can be downloaded as follows:

In [6]:
git clone git@github.com:databio/bedstat
git clone -b dev git@github.com:databio/bbconf

# Install Python dependencies
pip install piper --user
#pip install git+https://github.com/databio/bbconf.git@dev --user

# Install R dependencies
Rscript scripts/installRdeps.R

Cloning into 'bedstat'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 362 (delta 81), reused 106 (delta 43), pack-reused 197
Receiving objects: 100% (362/362), 57.94 KiB | 1.81 MiB/s, done.
Resolving deltas: 100% (155/155), done.
Cloning into 'bbconf'...
remote: Enumerating objects: 251, done.
remote: Counting objects: 100% (251/251), done.
remote: Compressing objects: 100% (178/178), done.
remote: Total 251 (delta 148), reused 154 (delta 61), pack-reused 0
Receiving objects: 100% (251/251), 42.52 KiB | 4.72 MiB/s, done.
Resolving deltas: 100% (148/148), done.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


We'll need to create a directory where we can store the stats and plots generated by `bedstat`. Additionally, we'll create a directory where we can store log and metadata files that we'll need later on.

In [7]:
mkdir bedstat/bedstat_output
mkdir bedstat/bedstat_pipeline_logs

In order to use bbconf, we'll need to create a minimal configuration.yaml file. The path to this configuration file can be stored in the environment variable `$BEDBASE`.

In [8]:
cat bedbase_demo_PEPs/bedbase_configuration.yaml

path:
  pipelines_output: ../bedstat/bedstat_output

database:
  host: localhost
  bed_index: bed_index
  bedset_index: bedset_index

server:
  host: 0.0.0.0
  port: 8000


### Inititiate a local elasticsearch cluster

In addition to generate statistics and plots, [bedstat](https://github.com/databio/bedstat) inserts JSON formatted metadata into an [elasticsearch](https://www.elastic.co/elasticsearch/?ultron=[EL]-[B]-[AMER]-US+CA-Exact&blade=adwords-s&Device=c&thor=elasticsearch&gclid=Cj0KCQjwjcfzBRCHARIsAO-1_Oq5mSdze16kripxT5_I__EeH9F-xUCz_khEvzGL7q_mqP62CahJ9SIaAg2BEALw_wcB) database that it'll later be used to search and extract files and information about them. (This step may have to be performed outside the notebook since these commands ask for a sudo password. 

In [10]:
# If docker is not already installed, you can do so with the following commands
#(make sure you have sudo permissions)

sudo apt-get update
sudo apt-get install docker-engine -y

# Create a persistent volume to house elastic search data
sudo docker volume create es-data

# Run the docker container for elasticsearch
sudo docker run -p 9200:9200 -p 9300:9300 -v es-data:/usr/share/elasticsearch/data -e "xpack.ml.enabled=false" \
  -e "discovery.type=single-node" elasticsearch:7.5.1

[sudo] password for jev4xy: 


### Run bedstat  on the demo PEP
To run [bedstat](https://github.com/databio/bedstat) and the other required pipelines in this demo, we will rely on the pipeline submission engine [looper](http://looper.databio.org/en/latest/),which can be installed in the following manner

In [ ]:
pip install --user loopercli

In order to establish a modular connection between a project and a pipeline, we'll need to create a [pipeline interface](http://looper.databio.org/en/latest/linking-a-pipeline/) file, which tells looper how to run the pipeline. If `bedstat` is being run from an HPC environment where docker is not available, we recommend running the pipeline using the `--no-db-commit` flag (this will only calculate statistics and generate plots but will not insert this information into the local elasticsearch cluster. Once we have generated plots and statistics, we can insert them into our local elasticsearch cluster running `bedstat` with the `--just-db-commit` flag. If your data lives on a local environment, as it's the case in this tutorial, it's not necessary to set those flags and we can run bedstat in the following manner:

In [15]:
#looper run bedbase_demo_PEPs/bedstat_config.yaml --no-db-commit --compute local --limit 1 -R

looper run bedbase_demo_PEPs/bedstat_config.yaml --bedbase-config bedbase_demo_PEPs/bedbase_configuration.yaml \
--no-db-commit --compute local -R

Command: run (Looper version: 0.12.4)
Reading sample table: '/home/jev4xy/Desktop/bedbase_tutorial/bedbase_demo_PEPs/bedstat_annotation_sheet.csv'
Activating compute package 'local'
Finding pipelines for protocol(s): bedstat
Known protocols: bedstat
'/home/jev4xy/Desktop/bedbase_tutorial/bedbase_demo_PEPs/../bedstat/pipeline/bedstat.py' appears to attempt to run on import; does it lack a conditional on '__main__'? Using base type: Sample
## [1 of 15] bedbase_demo_db1 (bedstat)
Submission settings lack memory specification
Writing script to /home/jev4xy/Desktop/bedbase_tutorial/bedstat/bedstat_pipeline_logs/submission/bedstat_bedbase_demo_db1.sub
Job script (n=1; 0.00 Gb): bedstat/bedstat_pipeline_logs/submission/bedstat_bedbase_demo_db1.sub
Compute node: cphg-51ksmr2
Start time: 2020-03-19 17:38:38
### Pipeline run code and environment:

*              Command:  `/home/jev4xy/Desktop/bedbase_tutorial/bedbase_demo_PEPs/../bedstat/pipeline/bedstat.py --bedfile bedbase_BEDfiles/GSE105587_

  - in 'y': chrCHR_HG107_PATCH, chrCHR_HG126_PATCH, chrCHR_HG1311_PATCH, chrCHR_HG1342_HG2282_PATCH, chrCHR_HG1362_PATCH, chrCHR_HG142_HG150_NOVEL_TEST, chrCHR_HG151_NOVEL_TEST, chrCHR_HG1832_PATCH, chrCHR_HG2021_PATCH, chrCHR_HG2023_PATCH, chrCHR_HG2030_PATCH, chrCHR_HG2058_PATCH, chrCHR_HG2063_PATCH, chrCHR_HG2066_PATCH, chrCHR_HG2072_PATCH, chrCHR_HG2095_PATCH, chrCHR_HG2104_PATCH, chrCHR_HG2116_PATCH, chrCHR_HG2191_PATCH, chrCHR_HG2213_PATCH, chrCHR_HG2217_PATCH, chrCHR_HG2232_PATCH, chrCHR_HG2233_PATCH, chrCHR_HG2235_PATCH, chrCHR_HG2239_PATCH, chrCHR_HG2247_PATCH, chrCHR_HG2288_HG2289_PATCH, chrCHR_HG2290_PATCH, chrCHR_HG2291_PATCH, chrCHR_HG2334_PATCH, chrCHR_HG26_PATCH, ch [... truncated]
4: In .Seqinfo.mergexy(x, y) :
  Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': chrUn_GL000224v1, chr17_GL000205v2_random, chrUn_GL000219v1, chrUn_GL000195v1, chrUn_GL000218v1, chr22_KI270733v1_random, chr1_KI270706v1_random, chrUn_GL000220v1, chrUn_GL000216v2

2: In .Seqinfo.mergexy(x, y) :
  Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': chrUn_GL000224v1, chrUn_KI270466v1, chrUn_KI270467v1
  - in 'y': chrCHR_HG107_PATCH, chrCHR_HG126_PATCH, chrCHR_HG1311_PATCH, chrCHR_HG1342_HG2282_PATCH, chrCHR_HG1362_PATCH, chrCHR_HG142_HG150_NOVEL_TEST, chrCHR_HG151_NOVEL_TEST, chrCHR_HG1832_PATCH, chrCHR_HG2021_PATCH, chrCHR_HG2023_PATCH, chrCHR_HG2030_PATCH, chrCHR_HG2058_PATCH, chrCHR_HG2063_PATCH, chrCHR_HG2066_PATCH, chrCHR_HG2072_PATCH, chrCHR_HG2095_PATCH, chrCHR_HG2104_PATCH, chrCHR_HG2116_PATCH, chrCHR_HG2191_PATCH, chrCHR_HG2213_PATCH, chrCHR_HG2217_PATCH, chrCHR_HG2232_PATCH, chrCHR_HG2233_PATCH, chrCHR_HG2235_PATCH, chrCHR_HG2239_PATCH, chrCHR_HG2247_PATCH, chrCHR_HG2288_HG2289_PATCH, chrCHR_HG2290_PATCH, chrCHR_HG2291_PATCH, chrCHR_HG2334_PATCH, chrCHR_HG26_PATCH, chrCHR_HG986_PATCH, chrCHR_HSCHR10_1_CTG1, chrCHR_HSCHR10_1_CTG2, chrCHR_HSCHR10_1_CTG4, chrCHR_HSCHR11_1_CTG1_2, chrCHR_HSCHR11_1_CTG5, chrCHR_HS

Finding overlaps...
Setting regionIDs...
jExpr: .N
Combining...
[1] "Plotting: /home/jev4xy/Desktop/bedbase_tutorial/bedstat/bedstat_output/a6a08126cb6f4b1953ba0ec8675df85a/GSE105977_ENCFF793SZW_conservative_idr_thresholded_peaks_GRCh38_chrombins"
Loading required namespace: BSgenome.Hsapiens.UCSC.hg38.masked
[1] "Plotting: /home/jev4xy/Desktop/bedbase_tutorial/bedstat/bedstat_output/a6a08126cb6f4b1953ba0ec8675df85a/GSE105977_ENCFF793SZW_conservative_idr_thresholded_peaks_GRCh38_gccontent"
promoterCore :	found 31
promoterProx :	found 59
exon :	found 156
intron :	found 1595
[1] "Plotting: /home/jev4xy/Desktop/bedbase_tutorial/bedstat/bedstat_output/a6a08126cb6f4b1953ba0ec8675df85a/GSE105977_ENCFF793SZW_conservative_idr_thresholded_peaks_GRCh38_partitions"
Warning messages:
1: In .Seqinfo.mergexy(x, y) :
  Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': chrUn_KI270587v1
  - in 'y': chrCHR_HG107_PATCH, chrCHR_HG126_PATCH, chrCHR_HG1311_PATCH, chrCHR_HG1342

<pre>
Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Loading required package: S4Vectors

...

<pre>
Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Loading required package: S4Vectors

*      Pipeline branch:  * master
*        Pipeline date:  2020-03-18 10:30:43 -0400

### Arguments passed to pipeline:

*     `bedbase_config`:  `bedbase_demo_PEPs/bedbase_configuration.yaml`
*            `bedfile`:  `bedbase_BEDfiles/GSM2827350_ENCFF928JXU_peaks_GRCh38.bed.gz`
*        `config_file`:  `bedstat.yaml`
*              `cores`:  `1`
*              `dirty`:  `False`
*       `force_follow`:  `False`
*    `genome_assembly`:  `hg38`
*              `input`:  `None`
*             `input2`:  `None`
*     `just_db_commit`:  `False`
*             `logdev`:  `False`
*                `mem`:  `4000`
*          `new_start`:  `False`
*       `no_db_commit`:  `True`
*      `output_parent`:  `bedstat/bedstat_pipeline_logs/results_pipeline`
*            `recover`:  `True`
*        `sample_name`:  `None`
*        `sample_yaml`:  `bedstat/bedstat_pipeline_logs/submission/bedbase_demo_db11.yaml`
*             `silent`:  `False`
*   `single_or_paired`:  `single`
*           `testmode`:  `Fals

*            Outfolder:  /home/jev4xy/Desktop/bedbase_tutorial/bedstat/bedstat_output/f41e12ddd3b6c4ee6da2140d0feee1ea/
*  Pipeline started at:   (03-19 17:42:28) elapsed: 0.0 _TIME_

### Version log:

*       Python version:  3.6.8
*          Pypiper dir:  `/home/jev4xy/.local/lib/python3.6/site-packages/pypiper`
*      Pypiper version:  0.12.1
*         Pipeline dir:  `/home/jev4xy/Desktop/bedbase_tutorial/bedstat/pipeline`
*     Pipeline version:  None
*        Pipeline hash:  bd90e7cbb5a8146fe95bce6c38548da519cb7602
*      Pipeline branch:  * master
*        Pipeline date:  2020-03-18 10:30:43 -0400

### Arguments passed to pipeline:

*     `bedbase_config`:  `bedbase_demo_PEPs/bedbase_configuration.yaml`
*            `bedfile`:  `bedbase_BEDfiles/GSE105587_ENCFF413ANK_peaks_hg19.bed.gz`
*        `config_file`:  `bedstat.yaml`
*              `cores`:  `1`
*              `dirty`:  `False`
*       `force_follow`:  `False`
*    `genome_assembly`:  `hg19`
*              `input`:  `None


    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Loading required package: S4Vectors

Attaching package: ‘S4Vectors’

The following object is masked from ‘package:base’:

    expand.grid

Loading required package: IRanges
Loading required package: GenomeInfoDb
[1] "Plotting: /home/jev4xy/Desktop/bedbase_tutorial/bedstat/bedstat_output/9dc6f420639e0a265f3f179b6b42713a/GSE105587_ENCFF809OOE_conservative_idr_thresholded_peaks_hg19_tssdist"
Done counting regionsGRL lengths.
Finding overlaps...
Setting regionIDs...
jExpr: .N
Combining...
[1] "Plotting: /home/jev

*         Compute host:  cphg-51ksmr2
*          Working dir:  /home/jev4xy/Desktop/bedbase_tutorial
*            Outfolder:  /home/jev4xy/Desktop/bedbase_tutorial/bedstat/bedstat_output/021461e2edc7b85315042c5ef80c7c03/
*  Pipeline started at:   (03-19 17:43:26) elapsed: 0.0 _TIME_

### Version log:

*       Python version:  3.6.8
*          Pypiper dir:  `/home/jev4xy/.local/lib/python3.6/site-packages/pypiper`
*      Pypiper version:  0.12.1
*         Pipeline dir:  `/home/jev4xy/Desktop/bedbase_tutorial/bedstat/pipeline`
*     Pipeline version:  None
*        Pipeline hash:  bd90e7cbb5a8146fe95bce6c38548da519cb7602
*      Pipeline branch:  * master
*        Pipeline date:  2020-03-18 10:30:43 -0400

### Arguments passed to pipeline:

*     `bedbase_config`:  `bedbase_demo_PEPs/bedbase_configuration.yaml`
*            `bedfile`:  `bedbase_BEDfiles/GSE105977_ENCFF634NTU_peaks_hg19.bed.gz`
*        `config_file`:  `bedstat.yaml`
*              `cores`:  `1`
*              `dirty`:  `F

In [16]:
#looper run bedbase_demo_PEPs/bedstat_config.yaml  --just-db-commit --compute local -R

looper run bedbase_demo_EPsP/bedstat_config.yaml --bedbase-config bedbase_demo_PEPs/bedbase_configuration.yaml \
--just-db-commit --compute local -R

Command: run (Looper version: 0.12.4)
Reading sample table: '/home/jev4xy/Desktop/bedbase_tutorial/bedbase_demo_PEPs/bedstat_annotation_sheet.csv'
Activating compute package 'local'
Finding pipelines for protocol(s): bedstat
Known protocols: bedstat
'/home/jev4xy/Desktop/bedbase_tutorial/bedbase_demo_PEPs/../bedstat/pipeline/bedstat.py' appears to attempt to run on import; does it lack a conditional on '__main__'? Using base type: Sample
## [1 of 15] bedbase_demo_db1 (bedstat)
Submission settings lack memory specification
Writing script to /home/jev4xy/Desktop/bedbase_tutorial/bedstat/bedstat_pipeline_logs/submission/bedstat_bedbase_demo_db1.sub
Job script (n=1; 0.00 Gb): bedstat/bedstat_pipeline_logs/submission/bedstat_bedbase_demo_db1.sub
Compute node: cphg-51ksmr2
Start time: 2020-03-19 17:51:53
Established connection with Elasticsearch: localhost
'id' metadata not available
'md5sum' metadata not available
'plots' metadata not available
'bedfile_path' metadata not available
Data: {'

## [6 of 15] bedbase_demo_db6 (bedstat)
Submission settings lack memory specification
Writing script to /home/jev4xy/Desktop/bedbase_tutorial/bedstat/bedstat_pipeline_logs/submission/bedstat_bedbase_demo_db6.sub
Job script (n=1; 0.00 Gb): bedstat/bedstat_pipeline_logs/submission/bedstat_bedbase_demo_db6.sub
Compute node: cphg-51ksmr2
Start time: 2020-03-19 17:51:55
Established connection with Elasticsearch: localhost
'id' metadata not available
'md5sum' metadata not available
'plots' metadata not available
'bedfile_path' metadata not available
Data: {'id': ['GSE91663_ENCFF319TPR_conservative_idr_thresholded_peaks_GRCh38'], 'gc_content': [0.507], 'regions_no': [17110], 'mean_absolute_TSS_dist': [51414986.6069], 'md5sum': ['9cd65cf4f07b83af35770c4a098fd4c6'], 'plots': [{'name': 'tssdist', 'caption': 'Region-TSS distance distribution'}, {'name': 'chrombins', 'caption': 'Regions distribution over chromosomes'}, {'name': 'gccontent', 'caption': 'GC content'}, {'name': 'partitions', 'caption

## [11 of 15] bedbase_demo_db11 (bedstat)
Submission settings lack memory specification
Writing script to /home/jev4xy/Desktop/bedbase_tutorial/bedstat/bedstat_pipeline_logs/submission/bedstat_bedbase_demo_db11.sub
Job script (n=1; 0.00 Gb): bedstat/bedstat_pipeline_logs/submission/bedstat_bedbase_demo_db11.sub
Compute node: cphg-51ksmr2
Start time: 2020-03-19 17:51:57
Established connection with Elasticsearch: localhost
Traceback (most recent call last):
  File "/home/jev4xy/Desktop/bedbase_tutorial/bedbase_demo_PEPs/../bedstat/pipeline/bedstat.py", line 59, in <module>
    with open(json_file_path, 'r', encoding='utf-8') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/home/jev4xy/Desktop/bedbase_tutorial/bedstat/bedstat_output/3e67ac88348d8b816a8ca50ab94eeade/GSM2827350_ENCFF928JXU_peaks_GRCh38.json'
## [12 of 15] bedbase_demo_db12 (bedstat)
Submission settings lack memory specification
Writing script to /home/jev4xy/Desktop/bedbase_tutorial/bedstat/bedstat_pipeline_l

After the previous steps have been executed, our BED files should be available for query on our local elastic search cluster. Files can be queried using the `bedbuncher` pipeline described in the below section. 


## Create bedsets using BEDBUNCHER

### Create a new PEP describing the bedset name and specific JSON query  
[bedbuncher](https://github.com/databio/bedbuncher) is a pipeline designed to create bedsets (sets of BED files retrieved from bedbase), with their respective statistics and additional outputs such as a `PEP` and an `iGD` database. In order to run `bedbuncher`, we will need to design an additional PEP describing the query as well as attributes such as the name assigned to the newly created bedset. This configuration file should point to the `JSON` file describing the query to find files of interest. The configuration file should have the following structure:

In [17]:
cat bedbase_demo_PEPs/bedbuncher_query.csv

sample_name,bedset_name,JSONquery_name,bbconfig_name,JSONquery_path,output_folder_path
bedset1,bedbase_demo_bedset,test_query,bedbase_configuration,source1,source2


In [20]:
cat bedbase_demo_PEPs/bedbuncher_config.yaml

metadata:
  sample_table: bedbuncher_query.csv
  output_dir: bedbuncher/bedbuncher_pipeline_logs
  pipeline_interfaces: ../bedbuncher/pipeline_interface.yaml 

derived_attributes: [JSONquery_path, bbconfig_path]
data_sources:
  source1: "bedbuncher/tests/{JSONquery_name}.json"
  source2: "bedbase_demo_PEPs/{bbconfig_name}.yaml"
constant_attributes:
  protocol: "bedbuncher"


###  Download bedbuncher  and install CML dependencies

To download the `bedbuncher` pipeline, simply clone the repository from github. Though not required, we'll also create a directory where we can store the pipeline logs. 

In [21]:
git clone git@github.com:databio/bedbuncher
mkdir bedbuncher/bedbuncher_pipeline_logs

Cloning into 'bedbuncher'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 235 (delta 22), reused 26 (delta 12), pack-reused 196
Receiving objects: 100% (235/235), 54.59 KiB | 1.71 MiB/s, done.
Resolving deltas: 100% (130/130), done.


One of the feats of `bedbuncher` includes [iGD](https://github.com/databio/iGD) database creation from the files in the bedset. `iGD` can be installed by cloning the repository from github, executing the make file to create the binary, and pointing the binary location with the `$PATH` environment variable. 

In [23]:
git clone git@github.com:databio/iGD
cd iGD
make

#Add iGD bin to PATH (might have to do this before starting the tutorial) Something like 
export PATH=$BEDBASEtutorial/iGD/bin/:$PATH
source ~/.bashrc

Cloning into 'iGD'...
remote: Enumerating objects: 367, done.
remote: Counting objects: 100% (367/367), done.
remote: Compressing objects: 100% (253/253), done.
remote: Total 1149 (delta 230), reused 242 (delta 111), pack-reused 782
Receiving objects: 100% (1149/1149), 18.57 MiB | 1.14 MiB/s, done.
Resolving deltas: 100% (651/651), done.
mkdir -p obj
mkdir -p bin
cc -c -g -O2 -lz -lm src/igd_base.c -o obj/igd_base.o 
src/igd_base.c: In function ‘get_fileinfo’:
src/igd_base.c:228:5: warning: ignoring return value of ‘fgets’, declared with attribute warn_unused_result [-Wunused-result]
     fgets(buf, 1024, fp);//head line
     ^~~~~~~~~~~~~~~~~~~~
src/igd_base.c:235:5: warning: ignoring return value of ‘fgets’, declared with attribute warn_unused_result [-Wunused-result]
     fgets(buf, 1024, fp);   //header
     ^~~~~~~~~~~~~~~~~~~~
src/igd_base.c: In function ‘get_igdinfo’:
src/igd_base.c:257:5: warning: ignoring return value of ‘fread’, declared with attribute warn_unused_result [-Wu

### Run bedbuncher using Looper 

Once we have cloned the `bedbuncher` repository, set our local elasticsearch cluster and created the `iGD` binary, we can run `bedbuncher` passing the location of the `bedbase` configuration file to the argument `--bedbase-config`. Note: if the path to the `bedbase` configration file has been stored in the `$BEDBASE` environment variable, it's not neccesary to pass the `--bedbase-config` argument. 

In [29]:
looper run  bedbase_demo_PEPs/bedbuncher_config.yaml  --bedbase-config bedbase_demo_PEPs/bedbase_configuration.yaml \
--compute local -R

Command: run (Looper version: 0.12.4)
Reading sample table: '/home/jev4xy/Desktop/bedbase_tutorial/bedbase_demo_PEPs/bedbuncher_query.csv'
Activating compute package 'local'
Finding pipelines for protocol(s): bedbuncher
Known protocols: bedbuncher
'/home/jev4xy/Desktop/bedbase_tutorial/bedbase_demo_PEPs/../bedbuncher/bedbuncher.py' appears to attempt to run on import; does it lack a conditional on '__main__'? Using base type: Sample
## [1 of 1] bedset1 (bedbuncher)
> Note (missing optional attribute): 'bedbuncher' requests sample attribute 'bbconfig_path' for option '--bedbase-config'
Writing script to /home/jev4xy/Desktop/bedbase_tutorial/bedbuncher/bedbuncher_pipeline_logs/submission/bedbuncher_bedset1.sub
Job script (n=1; 0.00 Gb): bedbuncher/bedbuncher_pipeline_logs/submission/bedbuncher_bedset1.sub
Compute node: cphg-51ksmr2
Start time: 2020-03-19 18:39:50
### Pipeline run code and environment:

*              Command:  `/home/jev4xy/Desktop/bedbase_tutorial/bedbase_demo_PEPs/../b

'bedbase_demo_bedset' summary info was successfully inserted into bedsets
Starting cleanup: 3 files; 0 conditional files for cleanup

Cleaning up flagged intermediate files. . .

### Pipeline completed. Epilogue
*        Elapsed time (this run):  0:00:00
*  Total elapsed time (all runs):  0:00:00
*         Peak memory (this run):  0 GB
*        Pipeline completed time: 2020-03-19 18:39:51

Looper finished
Samples valid for job generation: 1 of 1
Successful samples: 1 of 1
Commands submitted: 1 of 1
Jobs submitted: 1


##  Run local instance of the bedhost API

The last part of the tutorial consists on running a local instance of [bedhost](https://github.com/databio/bedhost/tree/master) (a REST API for bedstat and bedbuncher produced outputs) in order to explore plots, statistics and download pipeline outputs. To run `bedhost`, we can clone the github repository and pip install the package as follows:

In [30]:
git clone git@github.com:databio/bedhost
pip install bedhost/. --user

Cloning into 'bedhost'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 622 (delta 72), reused 66 (delta 29), pack-reused 511
Receiving objects: 100% (622/622), 178.95 KiB | 1.41 MiB/s, done.
Resolving deltas: 100% (405/405), done.
Processing ./bedhost
  Created wheel for bedhost: filename=bedhost-0.0.1-cp36-none-any.whl size=59799 sha256=3da5288dcd0f404fc58e0e4a6ebc382e7410619abd6657ff37a5fc0f8961709f
  Stored in directory: /tmp/pip-ephem-wheel-cache-h0h97mve/wheels/0d/13/b6/f9f990b04e991dfbb802fbdb6628b11149fedfb88a6916dfe0
Successfully built bedhost
  Found existing installation: bedhost 0.0.1
    Uninstalling bedhost-0.0.1:
      Successfully uninstalled bedhost-0.0.1
You should consider upgrading via the 'pip install --upgrade pip' command.


To start bedhost, we simply need to run the following commands passing the location of the `bedbase` config file to the `-c` flag.  

In [33]:
#Install bedhost dependencies
pip install itsdangerous --user

bedhost serve -c  bedbase_demo_PEPs/bedbase_configuration.yaml


You should consider upgrading via the 'pip install --upgrade pip' command.
DEBU 2020-03-19 18:45:30,471 | bedhost:est:263 > Configured logger 'bedhost' using logmuse v0.2.5 
DEBU 18:45:30 | bbconf:est:263 > Configured logger 'bbconf' using logmuse v0.2.5 
INFO 18:45:30 | bbconf:bbconf:58 > Established connection with Elasticsearch: localhost 
DEBU 18:45:30 | bbconf:bbconf:59 > Elasticsearch info:
{'name': '1ec537ca3e87', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'PamppPmESrKNFL1hqlo6gA', 'version': {'number': '7.5.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '3ae9ac9a93c95bd0cdc054951cf95d88e1e18d96', 'build_date': '2019-12-16T22:57:37.835892Z', 'build_snapshot': False, 'lucene_version': '8.3.0', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'} 
Traceback (most recent call last):
  File "/home/jev4xy/.local/bin/bedhost", line 8, in <module>
    sys.exit(main())
  File

: 1

If we have stored the path to the bedbase config in the environment variable `$BEDBASE` (suggested), it's not neccesary to pass the `-c` flag. 

In [ ]:
bedhost serve 